# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [30]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [31]:
weather_file = "../WeatherPy/output_data/cities.csv"
weather_file_df = pd.read_csv(weather_file)
weather_file_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,New Norfolk,-42.7826,147.0587,53.91,85,100,2.42,AU,1623897316
1,Rikitea,-23.1203,-134.9692,74.19,80,74,23.02,PF,1623897317
2,Port Hawkesbury,45.6169,-61.3485,62.55,99,75,4.00,CA,1623897318
3,Alyangula,-13.8483,136.4192,85.87,70,90,2.30,AU,1623897319
4,Puerto Ayora,-0.7393,-90.3518,73.54,89,33,4.00,EC,1623897319


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [32]:
gmaps.configure(api_key=g_key)

In [33]:
locations = weather_file_df[["Lat", "Lng"]].astype(float)
weight = weather_file_df["Humidity"].astype(float)

In [34]:
humidity_max = weather_file_df["Humidity"].max().astype(float)
humidity_max

100.0

In [35]:
fig = gmaps.figure(zoom_level=2, center=(29,114))

heat_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False, max_intensity=humidity_max, point_radius=2)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [38]:
all_indexes = weather_file_df[~((weather_file_df["Max Temp"] > 75) & (weather_file_df["Max Temp"] < 85) &
                                  ((weather_file_df["Cloudiness"] < 50) & (weather_file_df["Cloudiness"] > 15)) &
                                   (weather_file_df["Wind Speed"] < 10) & (weather_file_df["Humidity"] < 75))].index

weather_file_df.drop(all_indexes, inplace = True)

weather_file_df.dropna(how="any")

weather_file_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
146,Padang,-0.9492,100.3543,84.25,63,29,2.98,ID,1623897421
308,Marawi,7.9986,124.2928,79.65,62,46,2.13,PH,1623897542
330,Umm Hājar,13.2954,19.6966,81.95,50,29,6.69,TD,1623897559
374,Banjar,-8.1900,114.9675,84.06,65,39,3.33,ID,1623897591
479,Aksu,41.1231,80.2644,76.12,22,44,8.03,CN,1623897666


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [39]:
hotel_df = weather_file_df
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Address"] = ""

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address
146,Padang,-0.9492,100.3543,84.25,63,29,2.98,ID,1623897421,,
308,Marawi,7.9986,124.2928,79.65,62,46,2.13,PH,1623897542,,
330,Umm Hājar,13.2954,19.6966,81.95,50,29,6.69,TD,1623897559,,
374,Banjar,-8.1900,114.9675,84.06,65,39,3.33,ID,1623897591,,
479,Aksu,41.1231,80.2644,76.12,22,44,8.03,CN,1623897666,,


In [40]:
params = {"radius": 5000, "types": "lodging", "key": g_key}

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat}, {lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_detail = requests.get(base_url, params=params)
    
    hotel_detail = hotel_detail.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_detail["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = hotel_detail["results"][0]["vicinity"]
    
    except:
        print("Missing detail... skipping.")
    

Missing detail... skipping.


In [41]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address
146,Padang,-0.9492,100.3543,84.25,63,29,2.98,ID,1623897421,Mercure Padang,Jalan Purus IV No.8
308,Marawi,7.9986,124.2928,79.65,62,46,2.13,PH,1623897542,Derogongan Residence,"Sultan Omar Dianalan Boulevard, Marawi City"
330,Umm Hājar,13.2954,19.6966,81.95,50,29,6.69,TD,1623897559,,
374,Banjar,-8.1900,114.9675,84.06,65,39,3.33,ID,1623897591,The Hamsa Resort,"sing sing, Jalan Air Terjun, Cempaga"
479,Aksu,41.1231,80.2644,76.12,22,44,8.03,CN,1623897666,Pudong Holiday Hotel,"交通西路1号, Jiao Tong Lu, Akesu Shi, Akesu Diqu"


In [42]:
hotel_name = hotel_df["Hotel Name"].tolist()

In [43]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [44]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))